In [ ]:
import numpy as np
from sus.protocol_designer import System, Protocol, Potential, Compound_Protocol
from sus.protocol_designer.protocol import sequential_protocol
from IPython import display
from IPython.display import HTML
from quick_sim import setup_sim
from edward_tools.coupled_fq_potential import coupled_flux_qubit_non_linear_approx_pot, coupled_flux_qubit_non_linear_approx_force
import edward_tools.fq_runner as fq_runner
from edward_tools.visualization import animate_sim_flux_qubit
from edward_tools.initial_state_sampling import extra_constraint_00_and_11_only
# from PARAMETER_INPUT import *
# from PARAMETER_INPUT import _lambda, _theta, _eta


import kyle_tools as kt
import matplotlib.pyplot as plt

import importlib, os, datetime
from edward_tools import coupled_fq_protocol_library, cfq_runner
from edward_tools import coupled_fq_protocol_library
import edward_tools.cfq_batch_sweep as cfq_batch_sweep



coupled_fq_protocol_library = importlib.reload(coupled_fq_protocol_library)
create_system = coupled_fq_protocol_library.create_system
get_potential_shot_at_different_t = coupled_fq_protocol_library.get_potential_shot_at_different_t
get_potential_shot_at_different_t_1D = coupled_fq_protocol_library.get_potential_shot_at_different_t_1D
create_simple_protocol_parameter_dict = coupled_fq_protocol_library.create_simple_protocol_parameter_dict
coupled_fq_runner = importlib.reload(cfq_runner)
coupled_fq_protocol_library = importlib.reload(coupled_fq_protocol_library)
create_system = coupled_fq_protocol_library.create_system
get_potential_along_a_1D_cutline = coupled_fq_protocol_library.get_potential_along_a_1D_cutline
plotCutlines = coupled_fq_protocol_library.plotCutlines

In [ ]:
np.linspace(10, 200, 20)

In [ ]:
# 4. Szilard engine test

In [ ]:

PHI_0 = 2.067833848 * 1e-15
k_B = 1.38e-23
T = 0.5
k_BT = k_B * T

C_factor = 50
I_p_1, I_p_2 = 2e-6 , 2e-6  # Amp
I_m_1, I_m_2 = 7e-9, 7e-9                                # Amp
R_1, R_2 = 371, 371                                # ohm
C_1, C_2 = 4e-9 * C_factor, 4e-9 * C_factor                              # F
L_1, L_2 = 1e-9, 1e-9                              # H

quick_doubler = lambda x1, x2: np.hstack([np.array([x1] * 2), np.array([x2]*2)])
I_p, I_m = quick_doubler(I_p_1, I_p_2), quick_doubler(I_m_1, I_m_2)

m_c = C_1
m_1 = C_1
m_2 = C_2
x_c = PHI_0 / (2 * np.pi)
time_scale_factor = 1
t_c = np.sqrt(L_1 * C_1)


U0_1 = m_c * x_c**2 / t_c**2
U0_2 = m_2 * x_c**2 / t_c**2
kappa_1, kappa_2, kappa_3, kappa_4 = k_BT/U0_1, k_BT/U0_1, k_BT/U0_1, k_BT/U0_1

lambda_1 = 2 * np.sqrt(L_1 * C_1) / (C_1 * R_1)
theta_1  = 1
eta_1    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(2 * kappa_1 / 1**2)

lambda_2 = 2 * np.sqrt(L_1 * C_1) / (C_2 * R_2)
theta_2  = 1 / (C_2/C_1)
eta_2    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(2 * kappa_2 * (R_1 * C_1**2) / (R_2 * C_2**2))

lambda_3 = 2 * np.sqrt(L_1 * C_1) / (C_1 * R_1)
theta_3  = 4
eta_3    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(8 * kappa_3)

lambda_4 = 2 * np.sqrt(L_1 * C_1) / (C_2 * R_2)
theta_4  = 4 / (C_2/C_1)
eta_4    = np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(8 * kappa_4 * (R_1 * C_1**2) / (R_2 * C_2**2))

gamma = 5


beta_1 = 2 * np.pi * L_1 * I_p_1 / PHI_0; 
beta_2 = 2 * np.pi * L_2 * I_p_2 / PHI_0;

d_beta_1 = 2 * np.pi * L_1 * I_m_1 / PHI_0; 
d_beta_2 = 2 * np.pi * L_2 * I_m_2 / PHI_0;


_lambda = np.array([lambda_1, lambda_2, lambda_3, lambda_4])
_theta  = np.array([theta_1, theta_2, theta_3, theta_4])
_eta  =   np.array([eta_1, eta_2, eta_3, eta_4])

In [ ]:
eta_4

In [ ]:
np.sqrt(lambda_4 * kappa_4 * 4)

In [ ]:
np.sqrt(np.sqrt(L_1 * C_1)/ (R_1 * C_1)) * np.sqrt(2 * kappa_1 / 1**2)

In [ ]:
"""
# step 0: modify parameters
- All the parameters are stored in a separate file PARAMETER_INPUT
- You can override some of the parameters here.
"""
params = {}
params['N'] = 3_000
params['dt'] = 1/1_00
params['lambda'] = 1
params['beta'] = 1
params['sim_params'] = [_lambda, _theta, _eta]
params['target_work'] = None
params['comment'] = "test the effect of I_m asymmetry"



In [ ]:
"""
# step 1: Define potential
"""
coupled_fq_default_param = [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, x_c]
[phi_1_bound, phi_2_bound, phi_1dc_bound, phi_2dc_bound] = np.array([4, 4, 4, 4])/time_scale_factor

coupled_fq_domain = [[-phi_1_bound, -phi_2_bound, -phi_1dc_bound, -phi_2dc_bound], \
                     [phi_1_bound, phi_2_bound, phi_1dc_bound, phi_2dc_bound]]

# coupled_fq_pot = Potential(coupled_flux_qubit_pot_with_offset_at_0000, coupled_flux_qubit_force, 14, 4,\
#                            default_params = coupled_fq_default_param,  relevant_domain = coupled_fq_domain)

coupled_fq_pot = Potential(coupled_flux_qubit_non_linear_approx_pot, coupled_flux_qubit_non_linear_approx_force, 14, 4,\
                           default_params = coupled_fq_default_param,  relevant_domain = coupled_fq_domain)

In [ ]:
"""
# step 2: Define initial condition and protocol
"""
manual_domain=[np.array([-5, -5]), np.array([5, 5])]
# phi_1_dcx, phi_2_dcx = 3, 3
phi_1_dcx, phi_2_dcx = 0, 0
phi_1_dc, phi_2_dc = phi_1_dcx, phi_2_dcx

phi_1_dcx_off = 0
phi_2_dcx_off = 0
M_12_off = 0

# bookmark
phi_1_dcx_on = 3.0
phi_2_dcx_on = 3.0
M_12_on = -0.6

initial_parameter_dict = {
        "U0_1": U0_1,     "U0_2": U0_2,     "gamma_1": gamma,  "gamma_2": gamma,
        "beta_1": beta_1,   "beta_2": beta_2,   "d_beta_1": d_beta_1 ,   "d_beta_2": d_beta_2,
        "phi_1_x": 0,  "phi_2_x": 0,  "phi_1_dcx": phi_1_dcx_off,  "phi_2_dcx": phi_1_dcx_off,
        "M_12": M_12_off, 'x_c': x_c
}



In [ ]:
initial_parameter_dict

In [ ]:
zeroDissipation = False
params['sim_params'] = [_lambda, _theta, _eta]
if zeroDissipation:
    params['sim_params'] = [_lambda * 0, _theta, _eta * 0]



"""
1 <-> -1     (0, 0, 0), (0, 3, 0), (0, 0, 0)
2 <-> -2     (0, 3, 0), (0, 3, -0.05), (0, 3, 0)
3 <-> -3     (0, 3, -0.05), (0, 0, -0.05), (0, 3, -0.05)
4 <-> -4     (0, 0, -0.05), (0, 0, 0), (0, 0, -0.05)
5 <-> -5     (0, 0, 0), (3, 0, 0), (0, 0, 0)
6 <-> -6     (3, 0, 0), (0, 0, 0), (3, 0, 0)
"""
TR_initial_condition = [
    (phi_1_dcx_off, phi_2_dcx_off, M_12_off), 
    (phi_1_dcx_off, phi_2_dcx_on,  M_12_off), 
    (phi_1_dcx_off, phi_2_dcx_on,  M_12_on), 
    (phi_1_dcx_off, phi_2_dcx_off, M_12_on), 
    (phi_1_dcx_off, phi_2_dcx_off, M_12_off), 
    (phi_1_dcx_on,  phi_2_dcx_off, M_12_off)]

protocol_index = 0

# bookmark
# initial_parameter_dict["phi_1_dcx"], initial_parameter_dict["phi_2_dcx"], initial_parameter_dict["M_12"] = TR_initial_condition[protocol_index-1]
initial_parameter_dict["phi_1_dcx"], initial_parameter_dict["phi_2_dcx"], initial_parameter_dict["M_12"] = \
TR_initial_condition[0]

rest = lambda t: {"duration": t, "name":"rest"}
duration_t = 40
protocol_list = [
    # forward
    {"duration":duration_t, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on,  "M_12": M_12_off,  "name":"(1) mix in y direction"},
    {"duration":duration_t, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on,  "M_12": M_12_on,   "name":"(2) conditional tilt"},
    {"duration":duration_t, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_on,   "name":"(3) raise the barrier"},
    {"duration":duration_t, "phi_1_dcx": phi_1_dcx_on,  "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_on,  "name": "(4) conditional tilt in x"}, 
    {"duration":duration_t, "phi_1_dcx": phi_1_dcx_on,  "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_off,  "name":"(5) mix in x direction"}, 
    {"duration":duration_t, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_off,  "name":"(6) 4 well potential"}, 
    
]

protocol_time_array = [item["duration"] for item in protocol_list]
protocol_time_array.insert(0, 0)
protocol_time_array = np.cumsum(protocol_time_array)

print(params['sim_params'])
print(initial_parameter_dict["phi_1_dcx"], initial_parameter_dict["phi_2_dcx"], initial_parameter_dict["M_12"])
for x in protocol_list:
    print(x)

# print(protocol_list[0]["phi_1_dcx"], protocol_list[0]["phi_2_dcx"], protocol_list[0]["M_12"], )
# print(protocol_list[1]["phi_1_dcx"], protocol_list[1]["phi_2_dcx"], protocol_list[1]["M_12"], )
# print(protocol_list)

In [ ]:
np.array

In [ ]:

"""
# step 3: create the relevant storage protocol and computation protocol
"""
computation_protocol_parameter_dict = coupled_fq_protocol_library.customizedProtocol(initial_parameter_dict, \
                                                                    protocol_list)
storage_protocol, comp_protocol = create_system(computation_protocol_parameter_dict)

In [ ]:
"""
# step 4: create the coupled_fq_runner
"""
cfqr = cfq_runner.coupledFluxQubitRunner(potential = coupled_fq_pot, params = params, \
                                                storage_protocol= storage_protocol, \
                                                computation_protocol= comp_protocol)
cfqr.initialize_sim()
# cfqr.set_sim_attributes(extra_constraint=extra_constraint_00_and_11_only)
cfqr.set_sim_attributes()
init_state_saved = cfqr.init_state

In [ ]:
cfqr.show_initial_state()

# potential shape check

In [ ]:
min_U_along_cutline = []

fig, ax = plt.subplots(3, 4, figsize=[15, 12])
ax_flatten = ax.flatten()
X, Y, U, cutlineDirection, _targetAxis, _plotAxis = get_potential_along_a_1D_cutline(cfqr, t = 0, cutlineDirection = "v")
plt_line_data_at_t = plotCutlines(X, Y, U, cutlineDirection, cutlineValue = -3,\
                            contour_plt=ax_flatten[0], cutline_plt=ax_flatten[1], contours = 20,
                            showGraph=True)
min_U = np.min(U)
ax_flatten[1].set_title(f"t = 0 and 240, min_U = {min_U:.3g}")
min_U_along_cutline.append(min(plt_line_data_at_t['cutline_plot']['targetU']))


X, Y, U, cutlineDirection, _targetAxis, _plotAxis = get_potential_along_a_1D_cutline(cfqr, t = 40, cutlineDirection = "v")
plt_line_data_at_t = plotCutlines(X, Y, U, cutlineDirection, cutlineValue = -3,\
                            contour_plt=ax_flatten[2], cutline_plt=ax_flatten[3], contours = 20, 
                            showGraph=True)
min_U = np.min(U)
ax_flatten[3].set_title(f"t = 40, min_U = {min_U:.3g}")
min_U_along_cutline.append(min(plt_line_data_at_t['cutline_plot']['targetU']))

X, Y, U, cutlineDirection, _targetAxis, _plotAxis = get_potential_along_a_1D_cutline(cfqr, t = 80, cutlineDirection = "v")
plt_line_data_at_t = plotCutlines(X, Y, U, cutlineDirection, cutlineValue = -3,\
                            contour_plt=ax_flatten[4], cutline_plt=ax_flatten[5], contours = 20,  
                            showGraph=True)
min_U = np.min(U)
ax_flatten[5].set_title(f"t = 80, min_U = {min_U:.3g}")
min_U_along_cutline.append(min(plt_line_data_at_t['cutline_plot']['targetU']))

X, Y, U, cutlineDirection, _targetAxis, _plotAxis = get_potential_along_a_1D_cutline(cfqr, t = 120, cutlineDirection = "v")
plt_line_data_at_t = plotCutlines(X, Y, U, cutlineDirection, cutlineValue = -3,\
                            contour_plt=ax_flatten[6], cutline_plt=ax_flatten[7], contours = 20,  
                            showGraph=True)
min_U = np.min(U)
ax_flatten[7].set_title(f"t = 120, min_U = {min_U:.3g}")
min_U_along_cutline.append(min(plt_line_data_at_t['cutline_plot']['targetU']))


X, Y, U, cutlineDirection, _targetAxis, _plotAxis = get_potential_along_a_1D_cutline(cfqr, t = 160, cutlineDirection = "h")
plt_line_data_at_t = plotCutlines(X, Y, U, cutlineDirection, cutlineValue = 3,\
                             contour_plt=ax_flatten[8], cutline_plt=ax_flatten[9], contours = 20,  
                            showGraph=True)
min_U = np.min(U)
ax_flatten[9].set_title(f"t = 160, min_U = {min_U:.3g}")
min_U_along_cutline.append(min(plt_line_data_at_t['cutline_plot']['targetU']))

X, Y, U, cutlineDirection, _targetAxis, _plotAxis = get_potential_along_a_1D_cutline(cfqr, t = 200, cutlineDirection = "h")
plt_line_data_at_t = plotCutlines(X, Y, U, cutlineDirection, cutlineValue = 3,\
                             contour_plt=ax_flatten[10], cutline_plt=ax_flatten[11], contours = 20,  
                            showGraph=True)
min_U = np.min(U)
ax_flatten[11].set_title(f"t = 200, min_U = {min_U:.3g}")
min_U_along_cutline.append(min(plt_line_data_at_t['cutline_plot']['targetU']))


# plt.savefig("test_figure.png")
# X, Y, U, cutlineDirection, _targetAxis, _plotAxis = get_potential_along_a_1D_cutline(cfqr, t = 240, cutlineDirection = "h")
# plt_line_data_at_t = plotCutlines(X, Y, U, cutlineDirection, cutlineValue = 3, cutlineColor = 'orange',\
#                              contour_plt=ax_flatten[0], cutline_plt=ax_flatten[1], contours = 20,
#                             showGraph=True)

In [ ]:
min_U_along_cutline = []

fig, ax = plt.subplots(4, 4, figsize=[15, 15])
ax_flatten = ax.flatten()
X, Y, U, cutlineDirection, _targetAxis, _plotAxis = get_potential_along_a_1D_cutline(cfqr, t = 0, cutlineDirection = "v")
plt_line_data_at_t = plotCutlines(X, Y, U, cutlineDirection, cutlineValue = 0,\
                            contour_plt=ax_flatten[0], cutline_plt=ax_flatten[1], contours = 20,
                            showGraph=True)
min_U = np.min(U)
ax_flatten[1].set_title(f"t = 0 and 240, min_U = {min_U:.3g}")
min_U_along_cutline.append(min(plt_line_data_at_t['cutline_plot']['targetU']))


X, Y, U, cutlineDirection, _targetAxis, _plotAxis = get_potential_along_a_1D_cutline(cfqr, t = 40, cutlineDirection = "v")
plt_line_data_at_t = plotCutlines(X, Y, U, cutlineDirection, cutlineValue = 0,\
                            contour_plt=ax_flatten[2], cutline_plt=ax_flatten[3], contours = 20, 
                            showGraph=True)
min_U = np.min(U)
ax_flatten[3].set_title(f"t = 40, min_U = {min_U:.3g}")
min_U_along_cutline.append(min(plt_line_data_at_t['cutline_plot']['targetU']))

X, Y, U, cutlineDirection, _targetAxis, _plotAxis = get_potential_along_a_1D_cutline(cfqr, t = 80, cutlineDirection = "v")
plt_line_data_at_t = plotCutlines(X, Y, U, cutlineDirection, cutlineValue = 0,\
                            contour_plt=ax_flatten[4], cutline_plt=ax_flatten[5], contours = 20,  
                            showGraph=True)
min_U = np.min(U)
ax_flatten[5].set_title(f"t = 80, min_U = {min_U:.3g}")
min_U_along_cutline.append(min(plt_line_data_at_t['cutline_plot']['targetU']))

X, Y, U, cutlineDirection, _targetAxis, _plotAxis = get_potential_along_a_1D_cutline(cfqr, t = 120, cutlineDirection = "v")
plt_line_data_at_t = plotCutlines(X, Y, U, cutlineDirection, cutlineValue = 0,\
                            contour_plt=ax_flatten[6], cutline_plt=ax_flatten[7], contours = 20,  
                            showGraph=True)
min_U = np.min(U)
ax_flatten[7].set_title(f"t = 120, min_U = {min_U:.3g}")
min_U_along_cutline.append(min(plt_line_data_at_t['cutline_plot']['targetU']))


X, Y, U, cutlineDirection, _targetAxis, _plotAxis = get_potential_along_a_1D_cutline(cfqr, t = 160, cutlineDirection = "h")
plt_line_data_at_t = plotCutlines(X, Y, U, cutlineDirection, cutlineValue = 0,\
                             contour_plt=ax_flatten[8], cutline_plt=ax_flatten[9], contours = 20,  
                            showGraph=True)
min_U = np.min(U)
ax_flatten[9].set_title(f"t = 160, min_U = {min_U:.3g}")
min_U_along_cutline.append(min(plt_line_data_at_t['cutline_plot']['targetU']))

X, Y, U, cutlineDirection, _targetAxis, _plotAxis = get_potential_along_a_1D_cutline(cfqr, t = 200, cutlineDirection = "h")
plt_line_data_at_t = plotCutlines(X, Y, U, cutlineDirection, cutlineValue = 0,\
                             contour_plt=ax_flatten[10], cutline_plt=ax_flatten[11], contours = 20,  
                            showGraph=True)
min_U = np.min(U)
ax_flatten[11].set_title(f"t = 200, min_U = {min_U:.3g}")
min_U_along_cutline.append(min(plt_line_data_at_t['cutline_plot']['targetU']))

# X, Y, U, cutlineDirection, _targetAxis, _plotAxis = get_potential_along_a_1D_cutline(cfqr, t = 240, cutlineDirection = "h")
# plt_line_data_at_t = plotCutlines(X, Y, U, cutlineDirection, cutlineValue = 0, cutlineColor = 'orange',\
#                              contour_plt=ax_flatten[0], cutline_plt=ax_flatten[1], contours = 20,
#                             showGraph=True)

# actual simulations

In [ ]:
manual_domain=[np.array([-5, -5])/time_scale_factor, np.array([5, 5])/time_scale_factor]
mapping_state_1_to_state_2_dict = {"00": ["00", "10"], "01": ["00", "10"], "10": ["01", "11"], "11": ["01", "11"]}

In [ ]:
# step 5: perform simulations

simResult = cfq_batch_sweep.simulateSingleCoupledFluxQubit(params, initial_parameter_dict, protocol_list, \
                        potential = coupled_fq_pot, potential_default_param = coupled_fq_default_param,            
                        initial_state = init_state_saved, manual_domain = manual_domain, \
                        phi_1_dcx = phi_1_dcx,  phi_2_dcx = phi_2_dcx, \
                        percentage = 1, as_step = np.s_[::], measure_all_states=True)

In [ ]:
simResult["fidelity"]

In [ ]:
U0_1/k_BT

# animations

In [ ]:
# animations
vmin, vmax = 0, 100
frame_skip = 1000
# phi_1_dc, phi_2_dc = phi_1_dcx, phi_2_dcx
particle_number = 0
# all_state = simResult["cfqr"].sim.output.all_state['states'][particle_number:particle_number+1, :, :, :]
all_state = simResult["cfqr"].sim.output.all_state['states']
simResult["cfqr"].system.protocol_list = protocol_list
time_range = (computation_protocol_parameter_dict["t"][0], computation_protocol_parameter_dict["t"][-1])

ani,_,_ = animate_sim_flux_qubit(all_state, system = simResult["cfqr"].system ,
                                   times = time_range, frame_skip=frame_skip, color_by_state=True,
                                   vmin = vmin, vmax = vmax,
                                   manual_domain = manual_domain)

In [ ]:
# from IPython.display import HTML
from IPython import display
video = ani.to_html5_video()
html = display.HTML(video)
display.display(html)

# work analysis

In [ ]:
# %%notify
step_time_array = np.cumsum(np.array([x["duration"] for x in protocol_list]))/params['dt']
name_array = [x["name"] for x in protocol_list]


plt.hist(simResult["work_distribution"], bins = 45)
plt.show()
mean_work = np.mean(simResult["work_distribution"])
jarzyn_term = np.mean(np.exp(-
                             simResult["work_distribution"]))
print(f'jarzyn = {jarzyn_term}, mean work = {mean_work}')
print([x["duration"] for x in protocol_list])

In [ ]:
cfqr.system.protocol.get_params

In [ ]:


step_array = np.arange(simResult["work_statistic"][:,0].shape[0])
skip_step = int(len(step_array) * 0.01)
work_mean = simResult["work_statistic"][:,0]
work_std = simResult["work_statistic"][:,1]

coarse_step_array = step_array[::skip_step]
coarse_step_array = np.append(coarse_step_array, step_array[-1])
coarse_work_mean_array = work_mean[::skip_step]
coarse_work_mean_array = np.append(coarse_work_mean_array, work_mean[-1])
coarse_work_std_array = np.append(work_std[::skip_step], work_std[-1])

plt.errorbar(coarse_step_array, coarse_work_mean_array, yerr = coarse_work_std_array)
# plt.yscale('log')

for i, t in enumerate(step_time_array):
    plt.axvline(x = t, color = 'b', label = 'axvline - full height')
    # plt.text('2017-07-02 16:30:00',0.005,'BigNews1',rotation=90,va='top')
    plt.annotate(f"{i+1}", 
            xy=(t, 4.5),  
            # xytext=(quantile_75, 2.05*1e9),
            color='red', 
            ha='center', 
            size=16)



first_half = work_mean[0: len(work_mean)//2]
second_half = work_mean[len(work_mean)//2:][::-1]
abs_error = np.abs(first_half - second_half)
# plt.plot(step_array[0:len(step_array)//2], abs_error)
print(max(abs_error), min(abs_error))
print(f'jarzyn = {jarzyn_term}')
print([x["duration"] for x in protocol_list])

In [ ]:
cum_sum_index= np.cumsum(protocol_time_array)/params['dt']

In [ ]:
cum_sum_index[1]

In [ ]:
for i in range(len(protocol_time_array))[1:-1]:
    print(work_mean[int(cum_sum_index[i+1])-1] - work_mean[int(cum_sum_index[i]-1)])


In [ ]:
work_mean[7999] - work_mean[3999]

In [ ]:
# 100, w_mena = 1.37, jarzyn = 0.342
# 80, w_mena = 1.37, jarzyn = 0.342
# 40, w_mena = 1.36, jarzyn = 0.34
cfq_batch_sweep.saveSimulationResult(simResult, U0_1, timeOrStep = 'step', save = True, save_final_state = False)

In [ ]:
simResult['simulation_data']

In [ ]:
# crook analysis
# np.linspace(0.005, 0.2, 100)
test_w = np.linspace(-0.025, 0.0025, 21)
dw = 0.001
crook_analysis_result = []

for _w in test_w:
    array = simResult["work_distribution"]

    # The 0 index is to remove the single element array
    number_of_plus_W = len(np.where(np.logical_and(array <= _w + dw, array >= _w - dw))[0])
    number_of_minus_W = len(np.where(np.logical_and(array <= -_w + dw, array >= -_w - dw))[0])
    crook_analysis_result.append(np.array([_w, number_of_plus_W, number_of_minus_W * np.exp(_w)]))

crook_analysis_result = np.array(crook_analysis_result)    
print(f"dw = {dw}, dt = {params['dt']}, N = {params['N']}")
print("LHS: number of +W, RHS = number of -W * np.exp(+W)]")
all_w = crook_analysis_result[:, 0]
all_lhs = crook_analysis_result[:, 1]
all_rhs = crook_analysis_result[:, 2]
plt.xlabel("W")
plt.ylim([0, 2])
plt.plot(all_w, all_lhs/all_rhs, "*", label = "LHS")
# plt.plot(all_w, all_rhs, "-", label = "RHS")
plt.legend()
plt.show()
# print("p(W) = p(-W) exp(W)")
# print(f"LHS = {number_of_plus_W}, RHS = {number_of_minus_W * np.exp(test_w)}")
# print(f"mean work = {avg_W:.3f} and the std error of W = {error_W:.5f}")
# print(f'jarzyn = {jarzyn_term:.3f}, std error= {jarzyn_term_error:.5f}')
# print([x["duration"] for x in protocol_list])
# print(time_length_1, time_length_2, time_length_3, time_length_4, time_length_5, time_length_6)

In [ ]:
counts, bin_edges = np.histogram(simResult["work_distribution"], bins = np.linspace(-2, 2, 50),density=True)
counts, bin_edges = np.histogram(simResult["work_distribution"], bins = np.linspace(-0.01, 0.01, 20),density=True)

In [ ]:
log_P_ratio = np.log(counts / counts[::-1])

In [ ]:
plt.figure(figsize=(5, 5))
plt.plot(bin_edges[1:], bin_edges[1:], "--")
plt.scatter(bin_edges[:-1], log_P_ratio)
plt.xlabel("W")
plt.ylabel("log ratio")

In [ ]:
# crook analysis
# np.linspace(0.005, 0.2, 100)
test_w = np.linspace(-0.02, 0.02, 20)
dw = 0.001
crook_analysis_result = []

for _w in test_w:
    array = simResult["work_distribution"]

    # The 0 index is to remove the single element array
    number_of_plus_W = len(np.where(np.logical_and(array <= _w + dw, array >= _w - dw))[0])
    number_of_minus_W = len(np.where(np.logical_and(array <= -_w + dw, array >= -_w - dw))[0])
    crook_analysis_result.append(np.array([_w, number_of_plus_W, number_of_minus_W * np.exp(_w)]))

crook_analysis_result = np.array(crook_analysis_result)    
print(f"dw = {dw}, dt = {params['dt']}, N = {params['N']}")
print("LHS: number of +W, RHS = number of -W * np.exp(+W)]")
all_w = crook_analysis_result[:, 0]
all_lhs = crook_analysis_result[:, 1]
all_rhs = crook_analysis_result[:, 2]
plt.xlabel("W")
plt.plot(all_w, all_lhs, "x", label = "LHS")
plt.plot(all_w, all_rhs, "x", label = "RHS")
plt.legend()
plt.show()
# print("p(W) = p(-W) exp(W)")
# print(f"LHS = {number_of_plus_W}, RHS = {number_of_minus_W * np.exp(test_w)}")
print(f"mean work = {avg_W:.3f} and the std error of W = {error_W:.5f}")
print(f'jarzyn = {jarzyn_term:.3f}, std error= {jarzyn_term_error:.5f}')
print([x["duration"] for x in protocol_list])
print(time_length_1, time_length_2, time_length_3, time_length_4, time_length_5, time_length_6)

# fidelity analysis

In [ ]:
final_percentage_matrix

In [ ]:
final_percentage_matrix = np.array([d["final_percentage"] for d in simResult["fidelity"]]).T
text_box_array = ["\n".join(row) for row in final_percentage_matrix]

fig, ax = plt.subplots(1, 2, figsize = [10, 4])
# plt.figure(figsize=[4, 4])
# plt.title(f"N = {item['N']}, dt = {item['dt']}, t1 = {item['protocol_list'][-1]['duration']} ")
# _ax = ax[1]
# _ax.set_xlim(-5, 5);plt.ylim(-5, 5);
# _ax.text(-3.25, -3.5, text_box_array[0]); _ax.text(-3.25, 1.5, text_box_array[1])
# _ax.text(1.75, -3.5, text_box_array[2]); _ax.text(1.75, 1.5, text_box_array[3])
# _ax.vlines(0, ymin = -5, ymax = 5); _ax.hlines(0, xmin = -5, xmax = 5)

ideal_fidelity = np.array([['0.500', '0.000', '0.500', '0.000'], ['0.500', '0.000', '0.500', '0.000'],
                          ['0.000', '0.500', '0.000', '0.500'], ['0.000', '0.500', '0.000', '0.500']]).T
text_box_array = ["\n".join(row) for row in ideal_fidelity]
# plt.figure(figsize=[4, 4])
# plt.title(f"N = {item['N']}, dt = {item['dt']}, t1 = {item['protocol_list'][-1]['duration']} ")
_ax = ax[0]
_ax.set_xlim(-5, 5);plt.ylim(-5, 5);
_ax.text(-3.25, -3.5, text_box_array[0]); _ax.text(-3.25, 1.5, text_box_array[1])
_ax.text(1.75, -3.5, text_box_array[2]); _ax.text(1.75, 1.5, text_box_array[3])
_ax.vlines(0, ymin = -5, ymax = 5); _ax.hlines(0, xmin = -5, xmax = 5)

In [ ]:
import copy

In [ ]:
ideal_fidelity = copy.deepcopy(final_percentage_matrix)

In [ ]:
ideal_fidelity[0]["final_percentage"] = ['0.500', '0.000', '0.500', '0.000']
ideal_fidelity[1]["final_percentage"] = ['0.500', '0.000', '0.500', '0.000']
ideal_fidelity[2]["final_percentage"] = ['0.000', '0.500', '0.000', '0.500']
ideal_fidelity[3]["final_percentage"] = ['0.000', '0.500', '0.000', '0.500']

In [ ]:
ideal_fidelity

In [ ]:
_time_step_array = np.array(simResult["cfqr"].sim.output.all_state['step_indices']) * params['dt']
_phi_1dcx_time_array = []
_phi_2dcx_time_array = []
for _t in _time_step_array:
    params_at_t = cfqr.system.protocol.get_params(_t)
    _phi_1dcx_time_array.append(params_at_t[10])
    _phi_2dcx_time_array.append(params_at_t[11])

In [ ]:
all_state = simResult["cfqr"].sim.output.all_state['states']
_phi_1dc_array = all_state[:, :, 2, 0]
_phi_2dc_array = all_state[:, :, 3, 0]
_time_array = range(len(_phi_1dc_array))


In [ ]:
for line in _phi_2dc_array[0:10]:
    plt.plot(line)
plt.plot(_phi_2dcx_time_array, "--")
plt.title(r"$\varphi_{2dc}$ vs step ($\gamma$ " + f"= {gamma})"  )

In [ ]:
for line in _phi_1dc_array[::100]:
    plt.plot(line)
plt.plot(_phi_1dcx_time_array, "--")    
plt.title(r"$\varphi_{1dc}$ vs step ($\gamma$ " + f"= {gamma})"  )

In [ ]:
for line in _phi_1dc_array[0:10]:


In [ ]:
plt.plot(_phi_1dc_array[0:20])
plt.show()

In [ ]:
_phi_1dc_array